<a href="https://colab.research.google.com/github/malraharsh/Image-Captioning/blob/master/Image_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%%capture

!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip

!unzip Flickr8k_Dataset.zip

!mkdir textFiles
!unzip Flickr8k_text.zip -d /content/textFiles

!rm -r sample_data
!rm -r __MACOSX/

In [3]:
# import glob

# for i in glob.glob('*.txt'):
#     !rm $i

In [4]:
def load_file(filename):
    with open(filename, 'r') as file:
        text = file.read()
    return text

In [5]:
doc = load_file('/content/textFiles/Flickr8k.token.txt')

In [6]:
print(doc[:200])

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A lit


In [7]:
def load_descriptions(file):
    info = {}

    for line in file.split('\n'):
        if len(line) <= 1:
            continue
        
        img_id, img_descr = line.split('\t')

        img_id = img_id.split('.')[0]

        if img_id not in info:
            info[img_id] = list()
        info[img_id].append(img_descr)

    return info

In [8]:
descriptions = load_descriptions(doc)
list(descriptions.items())[:3]

[('1000268201_693b08cb0e',
  ['A child in a pink dress is climbing up a set of stairs in an entry way .',
   'A girl going into a wooden building .',
   'A little girl climbing into a wooden playhouse .',
   'A little girl climbing the stairs to her playhouse .',
   'A little girl in a pink dress going into a wooden cabin .']),
 ('1001773457_577c3a7d70',
  ['A black dog and a spotted dog are fighting',
   'A black dog and a tri-colored dog playing with each other on the road .',
   'A black dog and a white dog with brown spots are staring at each other in the street .',
   'Two dogs of different breeds looking at each other on the road .',
   'Two dogs on pavement moving toward each other .']),
 ('1002674143_1b742ab4b8',
  ['A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .',
   'A little girl is sitting in front of a large painted rainbow .',
   'A small girl in the grass plays with fingerpaints in front of a white canvas with a rainbow on it

In [9]:
# n = 3
# i = 0

# fix, ax = plt.subplots(n, 1, figsize=(25, 25))
# # ax.flatten()

# for (img_filename, img_descr) in list(descriptions.items())[:n]:
#     img_file = plt.imread('/content/Flicker8k_Dataset/' + img_filename + '.jpg')
#     ax[i].imshow(img_file)
#     print(' /n'.join(img_descr))
#     i += 1

In [10]:
# import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# print(stopwords.words('english'))

In [11]:
import string

def clean_sentence(sent):
    words = sent.split()
    words = map(str.lower, words)
    words = [w for w in words if len(w) > 1]
    words = [w for w in words if w.isalpha()] 
    return ' '.join(words)

def clean_descriptions(file):
    table = str.maketrans('', '', string.punctuation) #intab, outtab, toremove string, outtab replace intab

    for key, desc_list in descriptions.items():
        for idx, desc in enumerate(desc_list):
            desc_list[idx] = clean_sentence(desc)

clean_descriptions(descriptions)

In [12]:
descriptions['1001773457_577c3a7d70']

['black dog and spotted dog are fighting',
 'black dog and dog playing with each other on the road',
 'black dog and white dog with brown spots are staring at each other in the street',
 'two dogs of different breeds looking at each other on the road',
 'two dogs on pavement moving toward each other']

In [13]:
list(descriptions.items())[:1]

[('1000268201_693b08cb0e',
  ['child in pink dress is climbing up set of stairs in an entry way',
   'girl going into wooden building',
   'little girl climbing into wooden playhouse',
   'little girl climbing the stairs to her playhouse',
   'little girl in pink dress going into wooden cabin'])]

['scales', 'dc', 'joggers', 'paneled', 'trace']

In [15]:
import json

json.dump(descriptions, open('descriptions.txt', 'w'))
# json.load(descriptions, open('descriptions.txt', 'r'))

In [16]:
path_img_data = '/content/Flicker8k_Dataset/'
path_text = '/content/textFiles/'

In [18]:
train_images_path = load_file('/content/textFiles/Flickr_8k.trainImages.txt')
test_images_path = load_file('/content/textFiles/Flickr_8k.testImages.txt')

In [33]:
train_images_path[:100]

'2513260012_03d33305cf.jpg\n2903617548_d3e38d7f88.jpg\n3338291921_fe7ae0c8f8.jpg\n488416045_1c6d903fe0.j'

In [34]:
def create_images_list(imgnames, source=path_img_data):
    images = list()

    for img in imgnames.split('\n'):
        images.append(img)

    return images

train_images = create_images_list(train_images_path)
test_images = create_images_list(test_images_path)

In [97]:
from PIL import Image

img_shape = (299, 299)

def preprocess(img_path):
    img = image.load_img(img_path, target_size=img_shape)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [46]:
import glob
import pickle 
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [24]:
base_model = InceptionV3(weights='imagenet')

In [35]:
#!!!!!
n_sample = 100
train_images = train_images[:n_sample]
test_images = test_images[:n_sample]

In [27]:
model = Model(base_model.input, base_model.layers[-2].output)

In [108]:
def encode(img):
    try:
        img = preprocess(path_img_data + img)
        encoded_img = model.predict(img)
        encoded_img = np.squeeze(encoded_img)
        return encoded_img
    except:
        pass

encoded_train_images = list(map(encode, train_images))
encoded_test_images = list(map(encode, test_images))

In [112]:
encoded_train_images[0]

array([0.084479  , 0.0946759 , 0.09230016, ..., 0.07690614, 1.4561853 ,
       0.45119086], dtype=float32)

In [44]:
dir_path = 'Pickle'
if not os.path.isdir(dir_path):
    os.mkdir(dir_path)

In [113]:
def save_images_pkl(data, path):
    path = 'Pickle/' + path
    with open(path, 'wb') as file:
        pickle.dump(data, file)

def load_images_pkl(path): # not working
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

save_images_pkl(encoded_train_images, 'encoded_train_images.pkl')
save_images_pkl(encoded_test_images, 'encoded_test_images.pkl')

In [114]:
a = load_images_pkl('/content/Pickle/encoded_test_images.pkl')

In [115]:
list(a)[3]

array([0.50559866, 0.11145644, 0.61091816, ..., 0.05482279, 0.2874634 ,
       0.13505225], dtype=float32)

In [116]:
captions = np.reshape(list(descriptions.values()), -1) # 2D descr in flattened list of captions
len(captions)

40460

In [80]:
def show3(var):
    return list(var)[:3]

In [121]:
words_count = {}
word_count_thresh = 10

for line in captions:
    for word in line.split(' '):
        words_count[word] = words_count.get(word, 0) + 1 # 0 if no word

vocab = [w for w, count in words_count.items() if count >= word_count_thresh]        

In [125]:
print("Processed words from {} -> {}".format(len(words_count), len(vocab)))

Processed words from 8358 -> 1937


In [126]:
idx2word = {}
word2idx = {}

for idx, w in enumerate(vocab, 1):
    word2idx[w] = idx
    idx2word[idx] = w

In [128]:
size_vocab = len(vocab) + 1 # for appended 0
size_vocab

1938

In [129]:
# def create_vocab(descr): # no use
#     vocab = set()

#     for d_list in descr.values():
#         [vocab.update(d.split()) for d in d_list]

#     return vocab

# vocabulary = create_vocab(descriptions)
# list(vocabulary)[:5]

In [130]:
max_length_sent = max(map([split('/'), len], captions))

NameError: ignored